In [1]:
from HPC_classes import *

df , _ = SparkDataProcessor().process_data(
    data_csv_url= "https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/data/data.csv", 
    drop_col_yml_url= "https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/params.yml", 
    rename_col_json_url="https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/labels_relabel.json")

/DeepenData/.miniconda/envs/spark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/DeepenData/.miniconda/envs/spark/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/DeepenData/.miniconda/envs/spark/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The impli

23/05/28 23:19:50 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/05/28 23:19:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/28 23:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/28 23:19:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
my_target               = 'HOMA-IR altered'
it_targets  : list[str] = [my_target]
it_features : list[str] = ['Phe']

#df.drop(my_target, axis=1, inplace=True)

In [3]:
ray_data = ray.put(df)

2023-05-28 23:19:55,307	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [4]:
%%time

studies = make_multiple_studies(
    ray_data, 
    features = it_features,
    targets = it_targets,
    n_trials=1000
)

studies = ray.get(studies)

(pid=3657486) /DeepenData/.miniconda/envs/spark/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
(pid=3657486)   def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
(pid=3657486) /DeepenData/.miniconda/envs/spark/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-

CPU times: user 2.13 s, sys: 1.59 s, total: 3.72 s
Wall time: 2min 11s


(make_a_study pid=3657486) [I 2023-05-28 23:22:07,694] Trial 997 finished with values: [0.75, 0.0] and parameters: {'max_depth': 4, 'eta': 0.4134769344986083, 'subsample': 0.4375553255154464, 'lambda': 0.7474328870249592, 'alpha': 0.36154158551604954, 'scale_pos_weight': 4.047166057579743, 'kfold_splits': 3}. 
(make_a_study pid=3657486) [I 2023-05-28 23:22:07,724] Trial 996 finished with values: [0.9615384615384616, 0.0] and parameters: {'max_depth': 7, 'eta': 0.4513770997243157, 'subsample': 0.3535873163120441, 'lambda': 0.3119538282133658, 'alpha': 0.1259841859634736, 'scale_pos_weight': 3.378373704202806, 'kfold_splits': 4}. 
(make_a_study pid=3657486) [I 2023-05-28 23:22:07,732] Trial 998 finished with values: [0.4583333333333333, 0.0] and parameters: {'max_depth': 6, 'eta': 0.41460681965321, 'subsample': 0.27142594576431067, 'lambda': 0.7651033136095945, 'alpha': 0.33003688184839497, 'scale_pos_weight': 4.507705404009677, 'kfold_splits': 3}. 
(make_a_study pid=3657486) [I 2023-05-

In [7]:

make_pareto_plot(studies[0])

In [8]:
a_multi_study  = studies[0]
CUTOFF_AUC     = 0.9
CUTOFF_FEATURE = 0



bs_trials = []#pd.DataFrame()
ds_aucs   = []
for t in a_multi_study.get_trials():
        if t.values is None:
                continue
        if t.values[0] > CUTOFF_AUC and t.values[1] > CUTOFF_FEATURE:
                #bs_trials[t.number] = t.params#list(t.params.values()) #+ list(t.values)
                bs_trials.append(t.params)
                ds_aucs.append(t.values[0])



df_ranks_abnormal = pd.DataFrame()
df_ranks_healty   = pd.DataFrame()
AUCs              = []


for trial_num, one_trial in enumerate(bs_trials):


    kfold_splits, xgb_params = one_trial.pop('kfold_splits'), one_trial

    model_instance = ModelInstance(
                                        df  = df, 
                                        target      = it_targets[0], 
                                        test_size = 0.3, 
                                        xgb_params  = xgb_params, 
                                        kfold_splits = kfold_splits, 
                                        seed       = None)

    feature_metrics = model_instance.get_feature_explanation()
    df_ranks_abnormal[trial_num] = feature_metrics["SHAP_abnormal"]
    df_ranks_healty[trial_num]   = feature_metrics["SHAP_healty"]
    AUCs.append(model_instance.get_AUC_on_test_data())


import plotly.express as px

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index

df_ranks_long["auc_score"] = AUCs


fig_barplot = px.bar(
df_ranks_abnormal,
barmode="group",
)
def rank_sort(df_ranks_abnormal : pd.DataFrame) -> pd.Index:
        """This takes the dataframe, ranks its variables, and sorts them by the mean ranking"""
        return df_ranks_abnormal.rank(axis="rows").mean(axis="columns").sort_values(ascending=False).index

import plotly.express as px
order = rank_sort(df_ranks_abnormal) # Converts ranks to an ordered list

def wrapper_pcoorplot(df_ranks_abnormal, dims = order.tolist()):
        """Small wapper around plotly paralel coordinates plot"""
        fig_ranks = px.parallel_coordinates(
                df_ranks_abnormal.T, 
                dimensions=dims,
                color="Phe",
                #labels=labels_relabel,
                color_continuous_scale='BuGn',
        )

        fig_ranks.update_layout(
                yaxis_tickformat=".2f",
                coloraxis_colorbar=dict(title=f"{a_multi_study.study_name} (Shapley value)"),
        )

        return fig_ranks

# Creates the figure with the first [:15] features according to ranking
fig_abnormal = wrapper_pcoorplot(df_ranks_abnormal, order.tolist()[:10])
fig_abnormal.show() # Show fig, as this is a notebook

/tmp/ipykernel_3653465/684817265.py:38: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_3653465/684817265.py:39: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_3653465/684817265.py:38: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/tmp/ipykernel_3653465/684817265.py:39: Per